In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from preprocessing.logParser import openLogs

In [2]:
cursor = openLogs('logs/spbu_ACADRU_spring_2018-TL')

Описания таблиц и полей:
https://docs.google.com/document/d/12AQyASm_m0lV-aJQNmTCzIbfO2hj49unslcWMaxXjWY/edit# 

problem_check
cursor.executescript("""
        DROP TABLE IF EXISTS problem_check;
        CREATE TABLE problem_check 
        (
            user_id INT
            ,course_name varchar(300)
            ,execute_time varchar(100)
            ,page varchar(300)
            ,questions varchar(2000)
            ,grade real
            ,max_grade real
        )
    """)


     problem_events = ["edx.problem.hint.demandhint_displayed", "edx.problem.hint.feedback_displayed", "problem_check",
                      "problem_check_fail", "problem_graded", "problem_rescore", "problem_rescore_fail",
                      "problem_reset", "problem_save", "problem_show", "reset_problem", "reset_problem_fail",
                      "save_problem_fail", "save_problem_success", "showanswer"]
                      
https://docs.google.com/document/d/12AQyASm_m0lV-aJQNmTCzIbfO2hj49unslcWMaxXjWY/edit# 

In [3]:
#this request is for counting percentage (for each question) of people who succesfully passed a question (for each attempt)
#attempts, 
request = """
        select 
        user_id,         
        questions
        from problem_check
        where course_name = "course-v1:spbu+ACADRU+spring_2018"
        and questions NOT LIKE '__'        
        """.format("https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/b434df438c2045eba63afc089d64b3de/244a81ebf8a64b609cd8587bbe18edc5/")
cursor.execute(request)
data = cursor.fetchall()

columns_names = []
for i in cursor.description[:]:
    columns_names.append(i[0])

df = pd.DataFrame(data=data, columns=columns_names)
#df


In [4]:
print(df.head())

   user_id                                          questions
0   594202  {  "1dbdb694e1d84dd7b65d91eec160daa8_3_1":True...
1   594187  {  "1dbdb694e1d84dd7b65d91eec160daa8_3_1":True...
2   594202  {  "1dbdb694e1d84dd7b65d91eec160daa8_3_1":True...
3   594187  {  "b518db6fac834edea93e013d2e458987_3_1":True...
4   594202  {  "b518db6fac834edea93e013d2e458987_3_1":True...


In [5]:
#parsing of questions string in dataframe to t
list=pd.DataFrame(columns=('question','result'))

for row in df.loc[:,'questions']:
    row=(row[1:-1]).split(',')
    for cellRow in row:
        tmprow=cellRow.split(':')
        if tmprow[1]=='True':
            list=list.append(pd.DataFrame([[tmprow[0], 1]], columns=('question','result')))
        else:
            list=list.append(pd.DataFrame([[tmprow[0], 0]], columns=('question','result')))


In [6]:
print(list.head())

                                   question result
0    "1dbdb694e1d84dd7b65d91eec160daa8_3_1"      1
0    "1dbdb694e1d84dd7b65d91eec160daa8_8_1"      0
0    "1dbdb694e1d84dd7b65d91eec160daa8_4_1"      1
0   "1dbdb694e1d84dd7b65d91eec160daa8_10_1"      1
0    "1dbdb694e1d84dd7b65d91eec160daa8_6_1"      1
